<a href="https://colab.research.google.com/github/Jerry086/SALSA/blob/main/Colab/Precision_for_original_data_for_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The code serves as the benchmark for the precision@k for the system's performance. It calculates the precision@k for the original data without faiss, meanpool, PCA, etc.

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import time

In [2]:
train_metadata = pd.read_csv('/content/test_metadata_rootclass.csv')
train_embeddings = pd.read_csv('/content/test_embeddings.csv')

# Merge the metadata and embeddings on 'video_id'
train_df = pd.merge(train_metadata, train_embeddings, on='video_id')
train_df['root_classes'] = train_df['root_classes'].apply(eval)
train_df

,video_id,start_time_seconds,end_time_seconds,labels,root_classes,feature_0,feature_1,feature_2,feature_3,feature_4,...,feature_1270,feature_1271,feature_1272,feature_1273,feature_1274,feature_1275,feature_1276,feature_1277,feature_1278,feature_1279
0,iZcPr3wgJCw,30.0,40.0,['Engine knocking'],[Engine],132,228,153,42,232,...,118,255,29,89,0,0,65,185,255,222
1,iZmn0dUXP9E,40.0,50.0,"['Toothbrush', 'Electric toothbrush']","[Domestic sounds, home sounds]",80,123,108,0,156,...,46,255,93,222,0,0,3,185,155,118
2,iZ2K_GPtc6g,30.0,40.0,"['Sine wave', 'Chirp tone']",[Other sourceless],170,81,97,86,108,...,167,168,154,0,121,113,142,212,50,187
3,iZtF1lUPbEQ,20.0,30.0,"['Fart', 'Buzz', 'Music']","[Wild animals, Digestive, Music, Onomatopoeia]",181,113,129,155,154,...,0,62,116,150,0,50,93,0,143,0
4,iZFRfh1Wjt8,420.0,430.0,"['Vehicle', 'Vehicle horn, car horn, honking',...","[Alarm, Vehicle]",38,199,5,0,246,...,51,150,101,61,56,0,95,255,0,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21777,2Cmn2lplGfc,30.0,40.0,"['Propeller, airscrew']",[Vehicle],59,189,0,13,174,...,255,255,255,34,117,56,255,255,60,103
21778,2ClQcAd-wJg,30.0,40.0,"['Speech', 'Buzzer']","[Alarm, Human voice]",50,230,255,233,21,...,224,84,68,80,255,46,191,255,0,0
21779,-Sclnq22t7o,30.0,40.0,"['Burping, eructation']",[Digestive],23,105,204,22,146,...,0,145,0,111,105,133,47,141,13,174
21780,-SD9DkKyOrY,30.0,40.0,"['Percussion', 'Drum kit', 'Drum', 'Snare drum...",[Music],189,103,103,84,124,...,0,42,0,255,66,40,255,82,185,224


In [3]:
test_metadata = pd.read_csv('/content/testing_samples_id_based_on_class_size.csv')
test_embeddings = pd.read_csv('/content/test_embeddings.csv')

# Merge the metadata and embeddings on 'video_id'
test_df = pd.merge(test_metadata, test_embeddings, on='video_id')
test_df['root_classes'] = test_df['root_classes'].apply(eval)
test_df

,video_id,start_time_seconds,end_time_seconds,labels,root_classes,feature_0,feature_1,feature_2,feature_3,feature_4,...,feature_1270,feature_1271,feature_1272,feature_1273,feature_1274,feature_1275,feature_1276,feature_1277,feature_1278,feature_1279
0,FdeHk3CjgB0,0.0,10.0,"['Speech', 'Car', 'Glass', 'Breaking']","[Generic impact sounds, Glass, Vehicle, Human ...",74,100,186,79,239,...,255,218,138,227,255,0,103,255,0,62
1,WhI_e7hH99w,120.0,130.0,"['Speech', 'Glass']","[Glass, Human voice]",58,43,120,111,91,...,255,30,101,87,121,52,0,127,224,0
2,rBkHd299nY8,210.0,220.0,"['Music', 'Chink, clink']","[Glass, Music]",173,27,126,95,194,...,148,59,199,1,255,116,180,0,133,0
3,SVsz5CKZaUQ,160.0,170.0,['Shatter'],[Glass],94,77,163,138,72,...,136,182,102,163,75,112,93,87,104,85
4,fk6lhvFz-LA,150.0,160.0,['Glass'],[Glass],25,82,149,0,158,...,30,14,0,41,117,140,238,149,37,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3725,BGkXV9_wf9g,30.0,40.0,"['Speech', 'Animal', 'Domestic animals, pets',...","[Domestic animals, pets, Animal, Human voice]",83,79,167,65,145,...,73,44,71,179,208,103,126,67,213,255
3726,7gqGqbzLpZE,30.0,40.0,"['Animal', 'Domestic animals, pets', 'Dog', 'H...","[Wild animals, Domestic animals, pets, Animal]",76,45,142,55,78,...,80,110,88,129,135,154,125,208,189,198
3727,5KWTAq3GkfA,6.0,16.0,"['Animal', 'Domestic animals, pets', 'Cat', 'M...","[Domestic animals, pets, Animal]",59,102,169,30,137,...,114,117,121,126,126,130,114,133,130,187
3728,Lv-itTXoTiQ,500.0,510.0,"['Speech', 'Growling', 'Inside, small room']","[Wild animals, Domestic animals, pets, Acousti...",80,65,213,65,66,...,0,213,172,174,236,192,156,0,255,126


In [4]:
# Extract embeddings from dataframe
train_embeddings = train_df.iloc[:, 5:].values
train_labels = train_df['root_classes'].tolist()
test_embeddings = test_df.iloc[:, 5:].values
test_labels = test_df['root_classes'].tolist()
print(train_embeddings.shape)
print(test_embeddings.shape)

(21782, 1280)
(3730, 1280)


In [5]:
pip install scikit-learn

In [6]:
from sklearn.neighbors import NearestNeighbors

In [7]:
def cal_precisionK(query, neighbors):
    query_labels = set(query)
    relevant_cnt = 0
    for neigh in neighbors:
        neighbor_labels = set(neigh)
        if not query_labels.isdisjoint(neighbor_labels):
            relevant_cnt += 1
    return relevant_cnt / len(neighbors)

In [8]:
def sklearn_evaluation(k):
    nn_model = NearestNeighbors(n_neighbors=k)
    nn_model.fit(train_embeddings)
    distances, indices = nn_model.kneighbors(test_embeddings, n_neighbors=k)
    result_labels = []
    for idxs in indices:
        result_labels.append([train_labels[i] for i in idxs])
    return result_labels

In [9]:
query_results = sklearn_evaluation(100)
print(len(query_results))

3730


In [10]:
k_level = np.arange(5, 101, 5, dtype=int)
precision = defaultdict(list)

for k in k_level:
    for test_label, query_result in zip(test_labels, query_results):
        precision[k].append(cal_precisionK(test_label, query_result[:k]))

    print(f'Overall precision at {k} is:', np.mean(precision[k]))

Overall precision at 5 is: 0.5801072386058982
Overall precision at 10 is: 0.5071313672922252
Overall precision at 15 is: 0.4821626452189455
Overall precision at 20 is: 0.4674128686327077
Overall precision at 25 is: 0.45635388739946386
Overall precision at 30 is: 0.4489722966934763
Overall precision at 35 is: 0.4437916507085408
Overall precision at 40 is: 0.43886729222520104
Overall precision at 45 is: 0.4358117366696455
Overall precision at 50 is: 0.4335388739946381
Overall precision at 55 is: 0.4317036314891542
Overall precision at 60 is: 0.4294504021447721
Overall precision at 65 is: 0.42808414106001236
Overall precision at 70 is: 0.42690923018000765
Overall precision at 75 is: 0.42584092940125107
Overall precision at 80 is: 0.4249396782841823
Overall precision at 85 is: 0.424292698312569
Overall precision at 90 is: 0.4238039916592195
Overall precision at 95 is: 0.4231748271482997
Overall precision at 100 is: 0.4225951742627346
